# DEKAE: Dynamic Edge-driven Topology for Episodic Few-Shot Learning
## Extension of FSAKE with Supervised Relational Recovery

**Paper identity**: *A topology-learning framework with explicit structural regularization and supervised relational recovery for episodic few-shot learning.*

| Section | Description |
|---------|-------------|
| 1 | Environment Setup & GPU Check |
| 2 | Google Drive Mount & Data Pipeline |
| 3 | Dataset & Episodic Loader |
| 4 | Backbone (Conv4) |
| 5 | Static k-NN Graph Module (FSAKE Baseline) |
| 6 | Edge Incidence Matrix & Dynamic Edge Features |
| 7 | Adaptive Topology Reconstruction |
| 8 | Knowledge Filtering with Edge Awareness |
| 9 | Supervised Edge Loss (Support-Support Only) |
| 10 | Sparsity Regularization |
| 11 | Full Model Assembly (DEKAE) |
| 12 | Training Loop with Checkpointing |
| 13 | Evaluation & Statistical Testing |
| 14 | Synthetic Graph Recovery Experiment |
| 15 | Ablation Study Runner |
| 16 | Metrics Logging (W&B) |
| 17 | Visualization Suite |

## Section 1: Environment Setup & Dependencies

In [1]:
# ── Section 1: Environment Setup & Dependencies ──────────────────────────────
# Run once per Colab session. Installs torch-geometric matching the Colab CUDA version.

import sys
import torch
import os

# 1. Downgrade setuptools to avoid build errors with older packages
!pip install -q "setuptools<70" wheel Cython

# 2. Install standard dependencies
!pip install -q torch-geometric wandb scipy scikit-learn matplotlib seaborn networkx tqdm Pillow

# 3. Install learn2learn from source in editable mode (bypasses wheel build failure)
if not os.path.exists("learn2learn"):
    !git clone https://github.com/learnables/learn2learn.git

# Install with -e (editable) to skip bdist_wheel
!cd learn2learn && pip install -e .

print("PyTorch version :", torch.__version__)
print("CUDA available  :", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU             :", torch.cuda.get_device_name(0))
    print("VRAM            :", round(torch.cuda.get_device_properties(0).total_memory / 1e9, 2), "GB")
else:
    print("⚠ No GPU detected — switch Runtime → Change runtime type → GPU")

# ── Reproducibility ───────────────────────────────────────────────────────────
import random, numpy as np

GLOBAL_SEED = 42

def set_seed(seed: int = GLOBAL_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Obtaining file:///content/learn2learn
  Preparing metadata (setup.py) ... done
  Attempting uninstall: learn2learn
    Found existing installation: learn2learn 0.2.1
    Uninstalling learn2learn-0.2.1:
      Successfully uninstalled learn2learn-0.2.1
  Running setup.py develop for learn2learn
PyTorch version : 2.10.0+cu128
CUDA available  : True
GPU             : NVIDIA RTX PRO 6000 Blackwell Server Edition
VRAM            : 101.97 GB
Device: cuda


## Section 2: Google Drive Mount & Data Pipeline

In [2]:
# ── Section 2: Data Download + Google Drive Setup ────────────────────────────
# Strategy:
#   • Datasets are downloaded automatically via learn2learn (no manual work).
#   • Downloaded data lives on /content/data  (fast local SSD).
#   • Checkpoints & results are saved to Google Drive so they survive
#     session disconnects.
#
# Supported datasets (all download automatically on first run):
#   ACTIVE_DATASET = 'miniImageNet'   ← change to switch dataset
#   Other options : 'tieredImageNet' | 'cifarFS' | 'cub'

import shutil, pathlib, zipfile, sys, os

# ── Fix Import Shadowing ─────────────────────────────────────────────────────
# The repo is at /content/learn2learn. The package is /content/learn2learn/learn2learn.
# If CWD is /content, 'import learn2learn' finds the repo folder (namespace pkg) first.

# 1. Force reload if already loaded incorrectly
if "learn2learn" in sys.modules:
    import learn2learn
    # If it lacks __file__, it's likely the namespace package (broken)
    if not hasattr(learn2learn, "__file__") or learn2learn.__file__ is None:
        print("Creating fix: unloading broken learn2learn module...")
        del sys.modules["learn2learn"]

# 2. Prepend repo path to sys.path so the inner package is found first
if os.path.exists("/content/learn2learn"):
    if "/content/learn2learn" not in sys.path:
        sys.path.insert(0, "/content/learn2learn")

import learn2learn as l2l
print(f"learn2learn imported from: {getattr(l2l, '__file__', 'unknown')}")

# ─────────────────────────────────────────────────────────────────────────────

# SWITCHED TO cifarFS for reliability (miniImageNet download is often unstable)
ACTIVE_DATASET  = 'cifarFS'
LOCAL_DATA_ROOT = pathlib.Path("/content/data")
LOCAL_DATA_ROOT.mkdir(parents=True, exist_ok=True)

# ── Step 1: Download dataset via learn2learn ─────────────────────────────────
from learn2learn.vision.datasets import (
    MiniImagenet, TieredImagenet, CIFARFS, CUBirds200
)

from torchvision import transforms

MEAN = [0.4712, 0.4499, 0.4031]
STD  = [0.2726, 0.2634, 0.2794]

BASE_TRANSFORM = transforms.Compose([
    transforms.Resize(84),
    transforms.CenterCrop(84),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
AUG_TRANSFORM = transforms.Compose([
    transforms.RandomResizedCrop(84),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

DATASET_CLS = {
    'miniImageNet' : MiniImagenet,
    'tieredImageNet': TieredImagenet,
    'cifarFS'      : CIFARFS,
    'cub'          : CUBirds200,
}

print(f"Downloading / loading {ACTIVE_DATASET} …")
cls = DATASET_CLS[ACTIVE_DATASET]

# download=True fetches the data automatically on first run.
# Subsequent runs reuse the cached copy in LOCAL_DATA_ROOT.
try:
    train_dataset = cls(root=str(LOCAL_DATA_ROOT), mode='train',
                        download=True, transform=BASE_TRANSFORM)
    val_dataset   = cls(root=str(LOCAL_DATA_ROOT), mode='validation',
                        download=True, transform=BASE_TRANSFORM)
    test_dataset  = cls(root=str(LOCAL_DATA_ROOT), mode='test',
                        download=True, transform=BASE_TRANSFORM)

    # Wrap in MetaDataset to ensure labels_to_indices exists (needed for EpisodicSampler)
    train_dataset = l2l.data.MetaDataset(train_dataset)
    val_dataset   = l2l.data.MetaDataset(val_dataset)
    test_dataset  = l2l.data.MetaDataset(test_dataset)

    print(f"{ACTIVE_DATASET} — "
          f"train: {len(train_dataset.labels_to_indices)} classes, "
          f"val: {len(val_dataset.labels_to_indices)} classes, "
          f"test: {len(test_dataset.labels_to_indices)} classes")
    print(f"Data stored at : {LOCAL_DATA_ROOT}")

except Exception as e:
    print(f"\n❌ Error downloading {ACTIVE_DATASET}: {e}")
    print("Tip: If miniImageNet fails, try switching ACTIVE_DATASET to 'cifarFS' or 'tieredImageNet'.")
    raise e

# ── Step 2: Mount Google Drive for checkpoints / results ─────────────────────
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    DRIVE_ROOT = pathlib.Path("/content/drive/MyDrive/FSAKE_Project")
except Exception:
    # Running outside Colab (local dev) — use local fallback
    DRIVE_ROOT = pathlib.Path("/tmp/FSAKE_Project")
    print("Not running on Colab — Drive fallback at:", DRIVE_ROOT)

CKPT_DIR    = DRIVE_ROOT / "checkpoints"
RESULTS_DIR = DRIVE_ROOT / "results"
CKPT_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("CKPT dir    :", CKPT_DIR)
print("Results dir :", RESULTS_DIR)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

learn2learn imported from: /content/learn2learn/learn2learn/__init__.py
cifarFS — train: 64 classes, val: 16 classes, test: 20 classes
Data stored at : /content/data


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Not running on Colab — Drive fallback at: /tmp/FSAKE_Project
CKPT dir    : /tmp/FSAKE_Project/checkpoints
Results dir : /tmp/FSAKE_Project/results


## Section 3: Dataset & Episodic Loader

In [3]:
# ── Section 3: Dataset & Episodic Loader ─────────────────────────────────────
# Wraps the learn2learn datasets (loaded in Section 2) into a simple episodic
# sampler. Works with miniImageNet, tieredImageNet, CIFAR-FS, and CUB.
#
# Episodic protocol:
#   S = N_way × K_shot  support samples  (labeled)
#   Q = N_way × N_query query  samples  (unlabeled at inference)

import torch
import torch.nn.functional as F

# ── Generic episodic sampler wrapping any learn2learn dataset ─────────────────

class EpisodicSampler:
    """
    Wraps a learn2learn dataset (which exposes `labels_to_indices` dict)
    and samples N-way K-shot episodes on demand.

    Usage:
        sampler = EpisodicSampler(train_dataset, n_way=5, k_shot=1, n_query=15)
        s_imgs, s_lbl, q_imgs, q_lbl = sampler.sample()   # one episode

    Can also be called as a function (for compatibility with train/eval loops):
        s_imgs, s_lbl, q_imgs, q_lbl = sampler(n_way=5, k_shot=1, n_query=15)
    """

    def __init__(self, dataset, n_way: int = 5, k_shot: int = 1,
                 n_query: int = 15):
        self.dataset  = dataset
        self.n_way    = n_way
        self.k_shot   = k_shot
        self.n_query  = n_query

        # Build class → list[global_index] map
        if hasattr(dataset, 'labels_to_indices'):
            self.class_to_indices = {
                c: list(idxs)
                for c, idxs in dataset.labels_to_indices.items()
                if len(idxs) >= k_shot + n_query
            }
        elif hasattr(dataset, 'y'):
            import numpy as _np
            _labels = _np.array(dataset.y)
            self.class_to_indices = {
                int(c): list(_np.where(_labels == c)[0])
                for c in _np.unique(_labels)
                if len(_np.where(_labels == c)[0]) >= k_shot + n_query
            }
        else:
            raise ValueError("Dataset must expose .labels_to_indices or .y")

        self.classes = sorted(self.class_to_indices.keys())
        assert len(self.classes) >= n_way, (
            f"Only {len(self.classes)} classes available, N-way={n_way}")
        print(f"EpisodicSampler ready: {len(self.classes)} classes, "
              f"{n_way}-way {k_shot}-shot {n_query}-query")

    def sample(self):
        """Returns (support_imgs, support_labels, query_imgs, query_labels)."""
        episode_classes = random.sample(self.classes, self.n_way)
        s_imgs, s_labels, q_imgs, q_labels = [], [], [], []
        for local_lbl, cls in enumerate(episode_classes):
            pool = random.sample(self.class_to_indices[cls],
                                 self.k_shot + self.n_query)
            for i, idx in enumerate(pool):
                img, _ = self.dataset[idx]       # transform applied by dataset
                if not isinstance(img, torch.Tensor):
                    from torchvision.transforms.functional import to_tensor
                    img = to_tensor(img)
                if i < self.k_shot:
                    s_imgs.append(img);   s_labels.append(local_lbl)
                else:
                    q_imgs.append(img);   q_labels.append(local_lbl)
        return (torch.stack(s_imgs),  torch.tensor(s_labels, dtype=torch.long),
                torch.stack(q_imgs),  torch.tensor(q_labels, dtype=torch.long))

    def __call__(self, n_way=None, k_shot=None, n_query=None):
        """Allow calling as episode_fn(n_way, k_shot, n_query) for loop compat."""
        if n_way   is not None: self.n_way   = n_way
        if k_shot  is not None: self.k_shot  = k_shot
        if n_query is not None: self.n_query = n_query
        return self.sample()


# ── Instantiate samplers for all three splits ─────────────────────────────────
train_sampler = EpisodicSampler(train_dataset, n_way=5, k_shot=1, n_query=15)
val_sampler   = EpisodicSampler(val_dataset,   n_way=5, k_shot=1, n_query=15)
test_sampler  = EpisodicSampler(test_dataset,  n_way=5, k_shot=1, n_query=15)

# Augmented training sampler (same data, richer color/crop augmentation)
train_dataset_aug = DATASET_CLS[ACTIVE_DATASET](
    root=str(LOCAL_DATA_ROOT), mode='train',
    download=False, transform=AUG_TRANSFORM)
train_sampler_aug = EpisodicSampler(train_dataset_aug, n_way=5, k_shot=1, n_query=15)

# ── Quick episode sanity check ────────────────────────────────────────────────
_s_imgs, _s_lbl, _q_imgs, _q_lbl = train_sampler.sample()
print(f"Support : {_s_imgs.shape}  labels={_s_lbl.tolist()}")
print(f"Query   : {_q_imgs.shape}  labels={_q_lbl[:5].tolist()}…")

# ── Synthetic planted-partition episode generator (Section 2.3) ───────────────

def synthetic_episode(n_way=5, n_nodes_per_class=5, feat_dim=64,
                      sigma=0.8, device=DEVICE):
    """
    Generate a planted-partition episode for topology-recovery experiments.
    Returns node features X, labels y, and ground-truth adjacency A_star.
    sigma controls noise: larger sigma → harder topology recovery.
    """
    centers = torch.randn(n_way, feat_dim)
    center_dists = [
        (centers[i] - centers[j]).norm().item()
        for i in range(n_way)
        for j in range(i + 1, n_way)
    ]
    avg_sep = sum(center_dists) / len(center_dists)

    N = n_way * n_nodes_per_class
    X = torch.zeros(N, feat_dim)
    y = torch.zeros(N, dtype=torch.long)
    for c in range(n_way):
        sl = slice(c * n_nodes_per_class, (c + 1) * n_nodes_per_class)
        X[sl] = centers[c] + sigma * avg_sep * torch.randn(n_nodes_per_class, feat_dim)
        y[sl] = c

    # Ground-truth: intra-class = 1, inter-class = 0, no self-loops
    A_star = (y.unsqueeze(0) == y.unsqueeze(1)).float()
    A_star.fill_diagonal_(0)
    return X.to(device), y.to(device), A_star.to(device)


# ── Quick sanity check ────────────────────────────────────────────────────────
X_syn, y_syn, A_syn = synthetic_episode(n_way=5, n_nodes_per_class=4, feat_dim=64, sigma=0.6)
print(f"Synthetic X:{X_syn.shape}  y:{y_syn.shape}  A*:{A_syn.shape}")
print(f"Intra-class edges: {int(A_syn.sum().item())} "
      f"(expected {5 * 4 * 3} for 5 classes, 4 nodes each)")

EpisodicSampler ready: 64 classes, 5-way 1-shot 15-query
EpisodicSampler ready: 16 classes, 5-way 1-shot 15-query
EpisodicSampler ready: 20 classes, 5-way 1-shot 15-query


ValueError: Dataset must expose .labels_to_indices or .y

## Section 4: Backbone Network (Conv4)
Bit-for-bit identical to the FSAKE backbone. 128-dim output embedding.

In [ ]:
# ── Section 4: Backbone Network (Conv4) ──────────────────────────────────────
import torch.nn as nn

def conv_block(in_ch: int, out_ch: int) -> nn.Sequential:
    """Standard FSAKE conv block: Conv → BN → ReLU → MaxPool."""
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),
    )


class Conv4(nn.Module):
    """
    4-layer convolutional backbone identical to FSAKE.
    Input : (B, 3, 84, 84)
    Output: (B, embed_dim) — default 128 to match FSAKE paper.
    """

    def __init__(self, in_channels: int = 3, embed_dim: int = 128):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(in_channels, 64),   # → 42×42
            conv_block(64, 64),            # → 21×21
            conv_block(64, 64),            # → 10×10
            conv_block(64, 64),            # →  5× 5
        )
        self.proj = nn.Linear(64 * 5 * 5, embed_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = self.encoder(x)                          # (B, 64, 5, 5)
        h = h.view(h.size(0), -1)                    # (B, 1600)
        return self.proj(h)                          # (B, embed_dim)


# ── Sanity check ─────────────────────────────────────────────────────────────
_backbone = Conv4(embed_dim=128).to(DEVICE)
_dummy    = torch.randn(10, 3, 84, 84).to(DEVICE)
_out      = _backbone(_dummy)
print("Conv4 output shape:", _out.shape)   # expect (10, 128)
assert _out.shape == (10, 128), "Backbone shape mismatch!"
total_params = sum(p.numel() for p in _backbone.parameters() if p.requires_grad)
print(f"Conv4 trainable parameters: {total_params:,}")

## Section 5: Static k-NN Graph Module (FSAKE Baseline)

In [ ]:
# ── Section 5: Static k-NN Graph Module (FSAKE Baseline) ─────────────────────
# Builds A ∈ R^{N×N} from pairwise Euclidean distances; retains top-k per node.
# This is the graph construction used by FSAKE — plug-in swappable.

def build_knn_adjacency(node_feats: torch.Tensor, k: int,
                        symmetric: bool = True) -> torch.Tensor:
    """
    Args
    ----
    node_feats : (N, d) node embeddings
    k          : number of nearest neighbours per node
    symmetric  : if True, A = max(A, A^T) (undirected graph)

    Returns
    -------
    A : (N, N) float adjacency (0/1 values, no self-loops, row-normalised)
    """
    N = node_feats.size(0)
    # Pairwise squared Euclidean distance
    diff = node_feats.unsqueeze(0) - node_feats.unsqueeze(1)   # (N,N,d)
    dist = (diff ** 2).sum(-1)                                  # (N,N)

    # Mask self (set diagonal to large value so it is not selected in top-k)
    dist_masked = dist.clone()
    dist_masked.fill_diagonal_(float("inf"))

    # Keep k SMALLEST distances (nearest neighbours) → set all others to 0
    _, topk_idx = dist_masked.topk(k, dim=1, largest=False)
    A = torch.zeros(N, N, dtype=torch.float32, device=node_feats.device)
    A.scatter_(1, topk_idx, 1.0)

    if symmetric:
        A = torch.max(A, A.t())

    # Row-normalise (D^{-1} A)
    deg = A.sum(dim=1, keepdim=True).clamp(min=1e-6)
    A_norm = A / deg
    return A_norm


# ── Graph utility: density ────────────────────────────────────────────────────

def graph_density(A: torch.Tensor) -> float:
    """Fraction of possible directed edges that are non-zero (excl. diagonal)."""
    N = A.size(0)
    possible = N * (N - 1)
    actual   = (A > 0).float().sum().item() - (A.diagonal() > 0).float().sum().item()
    density  = actual / possible if possible > 0 else 0.0
    if density > 0.5:
        print(f"⚠  Graph density = {density:.3f} > 0.5 — risk of topology collapse!")
    return density


# ── Sanity check ─────────────────────────────────────────────────────────────
_feats = torch.randn(20, 128).to(DEVICE)
_A_knn = build_knn_adjacency(_feats, k=5)
d = graph_density(_A_knn)
print(f"k-NN adjacency shape: {_A_knn.shape}  density: {d:.3f}")

## Section 6: Edge Incidence Matrix & Dynamic Edge Features

**Key distinction from attention**: Attention produces a *scalar* weight per edge. Computing $E = BXW$ via the incidence matrix $B \in \{0,1\}^{N \times |E|}$ produces a *vector* $e_{ij} \in \mathbb{R}^d$ per edge — a $d$-dimensional signal retaining directional and heterogeneous relational information. This is equivalent to one step of line-graph convolution, where edges are first-class computational objects.

In [ ]:
# ── Section 6: Edge Incidence Matrix & Dynamic Edge Features ─────────────────
# B ∈ {0,1}^{N × |E|}  maps each edge to its two incident nodes.
# E = B^T X W  →  each edge gets a vector representation in R^d.

class EdgeIncidenceModule(nn.Module):
    """
    Builds the edge incidence matrix B from a given adjacency A,
    then computes edge features E = B^T X W using a shallow MLP.

    Parameters
    ----------
    in_dim  : node feature dimension d
    edge_dim: output edge feature dimension (default = in_dim for skip-compat.)
    hidden  : hidden dim of the 2-layer edge MLP (≤ 64 to avoid overfit)
    """

    def __init__(self, in_dim: int, edge_dim: int = None, hidden: int = 64):
        super().__init__()
        edge_dim = edge_dim or in_dim
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * in_dim, hidden, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, edge_dim, bias=True),
        )

    @staticmethod
    def adjacency_to_edge_index(A: torch.Tensor):
        """Return (src, dst) index arrays for all non-zero entries of A."""
        nz = A.nonzero(as_tuple=True)
        return nz[0], nz[1]         # src, dst

    def forward(self, X: torch.Tensor, A: torch.Tensor):
        """
        X : (N, d)  node features
        A : (N, N)  adjacency (may be weighted)

        Returns
        -------
        E         : (|E|, edge_dim)  vector edge features
        src, dst  : edge index tensors
        """
        src, dst = self.adjacency_to_edge_index(A)
        # Concatenate incident node features for each edge
        edge_input = torch.cat([X[src], X[dst]], dim=-1)    # (|E|, 2d)
        E = self.edge_mlp(edge_input)                        # (|E|, edge_dim)
        return E, src, dst


# ── Sanity check ─────────────────────────────────────────────────────────────
_eim   = EdgeIncidenceModule(in_dim=128, edge_dim=128, hidden=64).to(DEVICE)
_feats = torch.randn(20, 128).to(DEVICE)
_A     = build_knn_adjacency(_feats, k=5)
_E, _src, _dst = _eim(_feats, _A)
print(f"Edge features shape : {_E.shape}   (|E|={len(_src)}, edge_dim=128)")
print(f"src index range     : {_src.min().item()} – {_src.max().item()}")

## Section 7: Adaptive Topology Reconstruction

Low-rank bilinear scoring: $s_{ij} = (W_1 h_i)^\top (W_2 h_j)$ with rank $r \ll d$ bounds overfitting. Spectral normalization bounds $L_f$ (Lipschitz). Row-normalisation of $A'$ bounds $L_{MP}$. Together they ensure contraction (Proposition 3).

In [ ]:
# ── Section 7: Adaptive Topology Reconstruction ───────────────────────────────
from torch.nn.utils.parametrizations import spectral_norm as spectral_norm_wrap

class DynamicTopologyModule(nn.Module):
    """
    Learns A'(l) = f(H(l)) using low-rank bilinear scoring.

    s_ij = (W1 h_i)^T (W2 h_j) / sqrt(r)     [low-rank, rank r << d]

    Spectral normalization on W1,W2 → Lipschitz bound on f.
    Row-normalization of A'          → Lipschitz bound on MP.

    sparsity_mode ∈ {'none', 'l1', 'topk', 'laplacian'}
    """

    def __init__(self, feat_dim: int, rank: int = 16, edge_dim: int = 128,
                 sparsity_mode: str = "l1", topk: int = 5,
                 lambda_sparse: float = 0.01, edge_dropout: float = 0.1):
        super().__init__()
        self.rank          = rank
        self.sparsity_mode = sparsity_mode
        self.topk          = topk
        self.lambda_sparse = lambda_sparse
        self.edge_dropout  = edge_dropout

        # Low-rank projection (spectral-normalised)
        self.W1 = spectral_norm_wrap(nn.Linear(feat_dim, rank, bias=False))
        self.W2 = spectral_norm_wrap(nn.Linear(feat_dim, rank, bias=False))

        # Edge → A' scoring via edge feature dot-product
        self.edge_scorer = spectral_norm_wrap(
            nn.Linear(edge_dim, 1, bias=False))

    def forward(self, H: torch.Tensor, E: torch.Tensor,
                src: torch.Tensor, dst: torch.Tensor,
                sparsity_reg: bool = True):
        """
        H   : (N, d)      node features
        E   : (|E|, d_e)  edge features from EdgeIncidenceModule
        src,dst : edge indices

        Returns
        -------
        A_prime       : (N, N)  new normalised adjacency
        sparsity_loss : scalar  (0 if sparsity_reg=False)
        """
        N = H.size(0)

        # ── Low-rank bilinear scores ─────────────────────────────────────────
        scores_bilinear = (self.W1(H[src]) * self.W2(H[dst])).sum(-1) / (self.rank ** 0.5)

        # ── Edge-feature scores ──────────────────────────────────────────────
        scores_edge     = self.edge_scorer(E).squeeze(-1)

        edge_scores = scores_bilinear + scores_edge

        # ── Dropout on edge weights during training ──────────────────────────
        if self.training and self.edge_dropout > 0:
            mask = torch.bernoulli(
                torch.full_like(edge_scores, 1 - self.edge_dropout))
            edge_scores = edge_scores * mask

        # ── Soft adjacency via sigmoid ────────────────────────────────────────
        edge_weights = torch.sigmoid(edge_scores)

        # ── Build dense A' ────────────────────────────────────────────────────
        A_prime = torch.zeros(N, N, device=H.device, dtype=H.dtype)
        A_prime[src, dst] = edge_weights

        # ── Sparsity regularisation ───────────────────────────────────────────
        sparsity_loss = torch.tensor(0.0, device=H.device)
        if sparsity_reg:
            if self.sparsity_mode == "l1":
                sparsity_loss = self.lambda_sparse * A_prime.abs().mean()
            elif self.sparsity_mode == "topk":
                # Hard top-k mask (gradient stops here — ablation only)
                vals, idx = A_prime.topk(self.topk, dim=1)
                mask = torch.zeros_like(A_prime)
                mask.scatter_(1, idx, 1.0)
                A_prime = A_prime * mask.detach()
            elif self.sparsity_mode == "laplacian":
                # tr(H^T L H) where L = D - A'
                D = A_prime.sum(dim=1).diag()
                L = D - A_prime
                sparsity_loss = self.lambda_sparse * torch.trace(H.t() @ L @ H)

        # ── Row-normalise (Proposition 3 Lipschitz control) ──────────────────
        deg = A_prime.sum(dim=1, keepdim=True).clamp(min=1e-6)
        A_prime = A_prime / deg

        return A_prime, sparsity_loss


# ── Sanity check ─────────────────────────────────────────────────────────────
_dtm   = DynamicTopologyModule(feat_dim=128, rank=16, edge_dim=128).to(DEVICE)
_A_dyn, _sl = _dtm(_feats, _E, _src, _dst)
print(f"Dynamic A' shape: {_A_dyn.shape}  sparsity_loss={_sl.item():.4f}")
print(f"Graph density   : {graph_density(_A_dyn):.3f}")

## Section 8: Knowledge Filtering with Edge Awareness

In [ ]:
# ── Section 8: Knowledge Filtering with Edge Awareness ───────────────────────
# Node importance is now computed from aggregated *edge* representations
# rather than raw 1-hop degree counts (as in vanilla FSAKE).
# High-importance nodes gate the pooling step in the Graph U-Net path.

class EdgeAwareKnowledgeFilter(nn.Module):
    """
    Scores each node by aggregating its incident edge features, then
    applies a learnable gating MLP to produce a scalar importance score.

    importance_i = sigmoid( g( mean_{j: (i,j)∈E} e_{ij} ) )

    The scores are used for top-p node selection (Graph U-Net pooling analog).
    """

    def __init__(self, edge_dim: int, node_dim: int, hidden: int = 64):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(edge_dim + node_dim, hidden),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, 1),
        )

    def forward(self, H: torch.Tensor, E: torch.Tensor,
                src: torch.Tensor, A_prime: torch.Tensor) -> torch.Tensor:
        """
        H       : (N, d)
        E       : (|E|, edge_dim)
        src     : edge src indices
        A_prime : (N, N) row-normalised dynamic adjacency

        Returns
        -------
        scores  : (N,) importance scores in [0,1]
        H_pooled: (N, d) weighted node update  X' = A' E W'  via scatter
        """
        N = H.size(0)
        edge_dim = E.size(1)

        # Aggregate edge features per source node: mean of incident edges
        agg = torch.zeros(N, edge_dim, device=H.device)
        agg.scatter_add_(0, src.unsqueeze(-1).expand_as(E), E)
        counts = torch.zeros(N, device=H.device).scatter_add_(
            0, src, torch.ones(len(src), device=H.device))
        agg = agg / counts.unsqueeze(-1).clamp(min=1)

        # Gate: importance score from (aggregated edge features ∥ node feat)
        gate_input = torch.cat([agg, H], dim=-1)
        scores     = torch.sigmoid(self.gate(gate_input)).squeeze(-1)   # (N,)

        # Edge-to-node update: X' = A' · scatter(E) → message passing
        H_pooled = A_prime @ H                              # (N, d) simple MP
        H_pooled = H_pooled * scores.unsqueeze(-1)          # gate by importance

        return scores, H_pooled


# ── Sanity check ─────────────────────────────────────────────────────────────
_kf = EdgeAwareKnowledgeFilter(edge_dim=128, node_dim=128).to(DEVICE)
_scores, _H_pool = _kf(_feats, _E, _src, _A_dyn)
print(f"Node scores shape : {_scores.shape}")
print(f"H_pooled shape    : {_H_pool.shape}")
print(f"Score range       : [{_scores.min():.3f}, {_scores.max():.3f}]")

## Section 9: Supervised Edge Loss (Support-Support Only)

**Label-leakage prevention (Risk 5)**: The correction loss is computed **only on support-support pairs**. Query nodes never appear as supervision targets — they receive topology updates passively via message passing from the learned support graph.

$$\mathcal{L}_{edge} = \sum_{i,j \in \mathcal{S}} \mathbb{1}[y_i = y_j](1 - \cos(e_{ij})) + \mathbb{1}[y_i \neq y_j]\max(0, \cos(e_{ij}) - m)$$

In [ ]:
# ── Section 9: Supervised Edge Loss (Support-Support Only) ───────────────────

def edge_correction_loss(E: torch.Tensor,
                         src: torch.Tensor,
                         dst: torch.Tensor,
                         labels: torch.Tensor,
                         support_mask: torch.Tensor,
                         margin: float = 0.5) -> torch.Tensor:
    """
    Computes the edge-level contrastive correction loss.

    IMPORTANT: only support→support edges are used.
    An assertion guarantees no query indices leak into this computation.

    Parameters
    ----------
    E            : (|E|, d)  edge features from EdgeIncidenceModule
    src, dst     : edge endpoint indices (for ALL edges in the graph)
    labels       : (N,) — labels for support nodes; query labels NOT used
    support_mask : (N,) bool — True if node i is in the support set
    margin       : contrastive margin m

    Returns
    -------
    loss : scalar
    """
    # ── Strict label-leakage guard ────────────────────────────────────────────
    support_indices = support_mask.nonzero(as_tuple=True)[0]
    support_set     = set(support_indices.cpu().tolist())

    # Keep only support-support edges
    ss_mask = support_mask[src] & support_mask[dst]

    # ASSERT: no query node sneaks into the loss
    assert ss_mask.sum().item() <= (support_mask.sum() ** 2).item(), \
        "Edge loss applied to non-support edges — label leakage!"

    if ss_mask.sum().item() == 0:
        return torch.tensor(0.0, device=E.device, requires_grad=True)

    E_ss   = E[ss_mask]
    src_ss = src[ss_mask]
    dst_ss = dst[ss_mask]

    # Cosine similarity between edge endpoint features (proxy: edge features)
    cos_sim = F.cosine_similarity(E_ss[:, :E_ss.size(1)//2],
                                  E_ss[:, E_ss.size(1)//2:], dim=-1)
    # For the actual implementation use the edge vectors directly:
    # cos_sim = F.cosine_similarity(E_ss, ...) — simplified here

    same_class = (labels[src_ss] == labels[dst_ss]).float()

    pull_loss = same_class       * (1 - cos_sim)
    push_loss = (1 - same_class) * torch.clamp(cos_sim - margin, min=0)

    return (pull_loss + push_loss).mean()


# ── Sanity check ─────────────────────────────────────────────────────────────
# Simulate 5-way 1-shot (5 support) + 15 query nodes
_N_total   = 20
_n_support = 5
_labels_s  = torch.arange(_n_support).to(DEVICE)              # support labels
_labels    = torch.cat([_labels_s,
                        torch.zeros(_N_total - _n_support, dtype=torch.long).to(DEVICE)])
_sup_mask  = torch.zeros(_N_total, dtype=torch.bool).to(DEVICE)
_sup_mask[:_n_support] = True

_loss_edge = edge_correction_loss(_E, _src, _dst, _labels, _sup_mask)
print(f"Edge correction loss (support-only): {_loss_edge.item():.4f}")

## Section 10: Sparsity Regularization

Three togglable options — L1 (default, preserves variable degree), Laplacian smoothness, and top-k hard mask (ablation only). `graph_density()` raises a warning above 0.5 to detect collapse.

In [ ]:
# ── Section 10: Sparsity Regularization ──────────────────────────────────────
# Standalone helpers (also used inside DynamicTopologyModule).

def l1_sparsity_loss(A: torch.Tensor, lambda_: float = 0.01) -> torch.Tensor:
    """L1 regularisation on adjacency. Promotes sparse connectivity."""
    return lambda_ * A.abs().mean()


def laplacian_smoothness_loss(A: torch.Tensor, H: torch.Tensor,
                               lambda_: float = 0.01) -> torch.Tensor:
    """Laplacian smoothness: tr(H^T L H). Encourages connected nodes to agree."""
    D = A.sum(dim=1).diag()
    L = D - A
    return lambda_ * torch.trace(H.t() @ L @ H)


def topk_mask(A: torch.Tensor, k: int) -> torch.Tensor:
    """Hard top-k per row (ablation variant E3 ONLY — enforces fixed degree)."""
    # NOTE: gradient does not flow through this mask.
    _, idx = A.topk(k, dim=1)
    mask = torch.zeros_like(A)
    mask.scatter_(1, idx, 1.0)
    return A * mask.detach()


def graph_metrics(A: torch.Tensor) -> dict:
    """
    Returns a metrics dict for a single graph:
      graph_density, avg_degree, degree_std, edge_entropy.
    Raises a warning if density > 0.5.
    """
    N   = A.size(0)
    # Remove diagonal for degree counting
    A_no_diag = A.clone()
    A_no_diag.fill_diagonal_(0)

    degree    = (A_no_diag > 0).float().sum(dim=1)      # (N,)
    n_edges   = (A_no_diag > 0).float().sum().item()
    density   = n_edges / max(N * (N - 1), 1)

    # Edge entropy: treat each edge weight as a probability
    weights   = A_no_diag[A_no_diag > 0]
    p         = weights / weights.sum().clamp(min=1e-9)
    edge_ent  = -(p * (p + 1e-9).log()).sum().item()

    if density > 0.5:
        print(f"⚠  graph_density={density:.3f} > 0.5 — potential collapse")

    return {
        "graph_density": density,
        "avg_degree"   : degree.mean().item(),
        "degree_std"   : degree.std().item(),
        "edge_entropy" : edge_ent,
    }


# ── Demo ──────────────────────────────────────────────────────────────────────
_metrics = graph_metrics(_A_dyn)
print("Graph metrics:", {k: round(v, 4) for k, v in _metrics.items()})
print("L1 loss      :", l1_sparsity_loss(_A_dyn).item())

## Section 11: Full Model Assembly (DEKAE)

Forward pass: `node features + B → edge features E → dynamic A' → node update X' → knowledge filtering → skip connections`.

Warm-up schedule: static k-NN topology for first `T_warm` epochs. Curriculum `λ_edge`: linearly ramps from 0 to `lambda_edge` over first 20 % of training.

In [ ]:
# ── Section 11: Full Model Assembly (DEKAE) ───────────────────────────────────

class DEKAEModel(nn.Module):
    """
    Dynamic Edge-driven topology for Episodic few-shot learning (DEKAE).

    Architecture per GNN layer
    --------------------------
    1. EdgeIncidenceModule   : X, A(l) → E(l), src, dst
    2. DynamicTopologyModule : H(l), E(l) → A'(l), sparsity_loss
    3. EdgeAwareKnowledgeFilter : H(l), E(l), A'(l) → scores, H(l+1)
    4. Skip connection       : H(l+1) ← H(l+1) + H(0)

    Parameters
    ----------
    embed_dim     : backbone output dimension
    n_gnn_layers  : number of dynamic GNN layers (default 3)
    rank          : rank for low-rank bilinear scorer
    sparsity_mode : 'l1' | 'topk' | 'laplacian' | 'none'
    use_dynamic   : if False, fall back to static k-NN (FSAKE mode)
    knn_k         : k for static k-NN fallback
    """

    def __init__(self, embed_dim: int = 128, n_gnn_layers: int = 3,
                 rank: int = 16, sparsity_mode: str = "l1",
                 lambda_sparse: float = 0.01, lambda_edge: float = 0.5,
                 topk_k: int = 5, n_way: int = 5, use_dynamic: bool = True,
                 edge_dropout: float = 0.1):
        super().__init__()
        self.embed_dim     = embed_dim
        self.n_gnn_layers  = n_gnn_layers
        self.n_way         = n_way
        self.use_dynamic   = use_dynamic
        self.knn_k         = topk_k
        self.lambda_edge   = lambda_edge

        self.backbone = Conv4(embed_dim=embed_dim)

        self.edge_modules   = nn.ModuleList([
            EdgeIncidenceModule(embed_dim, embed_dim, hidden=64)
            for _ in range(n_gnn_layers)
        ])
        self.dynamic_modules = nn.ModuleList([
            DynamicTopologyModule(embed_dim, rank, embed_dim,
                                  sparsity_mode, topk_k, lambda_sparse,
                                  edge_dropout)
            for _ in range(n_gnn_layers)
        ])
        self.kf_modules = nn.ModuleList([
            EdgeAwareKnowledgeFilter(embed_dim, embed_dim)
            for _ in range(n_gnn_layers)
        ])

        self.classifier = nn.Linear(embed_dim, n_way)

    # ── Warm-up helpers ───────────────────────────────────────────────────────
    def set_use_dynamic(self, flag: bool):
        self.use_dynamic = flag

    # ── Forward ───────────────────────────────────────────────────────────────
    def forward(self, imgs_support: torch.Tensor, labels_support: torch.Tensor,
                imgs_query: torch.Tensor, lambda_edge_scale: float = 1.0):
        """
        imgs_support : (N_s, 3, 84, 84)
        labels_support : (N_s,)
        imgs_query   : (N_q, 3, 84, 84)
        lambda_edge_scale : curriculum multiplier [0,1] for edge loss

        Returns
        -------
        logits   : (N_q, n_way)
        aux_loss : edge correction loss + sparsity losses
        metrics  : dict with graph quality statistics
        """
        N_s = imgs_support.size(0)
        N_q = imgs_query.size(0)

        # ── 1. Encode all nodes ───────────────────────────────────────────────
        all_imgs  = torch.cat([imgs_support, imgs_query], dim=0)   # (N_s+N_q, ...)
        H0        = self.backbone(all_imgs)                         # (N, d)

        # Support mask (for edge loss — query nodes excluded)
        support_mask = torch.zeros(N_s + N_q, dtype=torch.bool,
                                   device=H0.device)
        support_mask[:N_s] = True

        H = H0
        total_sparsity_loss = torch.tensor(0.0, device=H0.device)
        total_edge_loss     = torch.tensor(0.0, device=H0.device)
        layer_stability     = []           # ||H(l+1) - H(l)||_F for Prop. 3

        for l in range(self.n_gnn_layers):
            # ── 2. Build initial adjacency (warm-up: static; else: expand) ───
            A_init = build_knn_adjacency(H, k=self.knn_k)

            # ── 3. Edge features ──────────────────────────────────────────────
            E, src, dst = self.edge_modules[l](H, A_init)

            if self.use_dynamic:
                # ── 4. Dynamic topology ───────────────────────────────────────
                A_prime, sp_loss = self.dynamic_modules[l](H, E, src, dst)
                total_sparsity_loss = total_sparsity_loss + sp_loss

                # ── 5. Edge correction loss (support-support only) ─────────────
                labels_full = torch.full((N_s + N_q,), -1, dtype=torch.long,
                                         device=H.device)
                labels_full[:N_s] = labels_support
                e_loss_l = edge_correction_loss(E, src, dst, labels_full,
                                                support_mask)
                total_edge_loss = total_edge_loss + e_loss_l * lambda_edge_scale
            else:
                A_prime = A_init         # static fallback (warm-up / FSAKE mode)

            # ── 6. Knowledge filtering & node update ──────────────────────────
            _, H_new = self.kf_modules[l](H, E, src, A_prime)
            H_new    = H_new + H0                                # skip connection

            # Layer representation stability diagnostic (Proposition 3)
            with torch.no_grad():
                stability = (H_new - H).norm().item()
            layer_stability.append(stability)

            H = H_new

        # ── 7. Classify query nodes via prototype cosine similarity ───────────
        H_support = H[:N_s]                                   # (N_s, d)
        H_query   = H[N_s:]                                   # (N_q, d)

        # Compute class prototypes
        prototypes = torch.zeros(self.n_way, self.embed_dim, device=H.device)
        for c in range(self.n_way):
            mask_c      = labels_support == c
            prototypes[c] = H_support[mask_c].mean(0)

        # Cosine similarity logits
        H_q_norm   = F.normalize(H_query, dim=-1)
        proto_norm = F.normalize(prototypes, dim=-1)
        logits     = H_q_norm @ proto_norm.t()                 # (N_q, n_way)

        aux_loss = (total_edge_loss * self.lambda_edge
                    + total_sparsity_loss)

        metrics = graph_metrics(A_prime)
        metrics["layer_stability"] = layer_stability

        return logits, aux_loss, metrics


# ── Sanity check ─────────────────────────────────────────────────────────────
_model = DEKAEModel(n_way=5, embed_dim=128).to(DEVICE)
_s_imgs = torch.randn(5, 3, 84, 84).to(DEVICE)       # 5-way 1-shot support
_s_lbl  = torch.arange(5).to(DEVICE)
_q_imgs = torch.randn(15, 3, 84, 84).to(DEVICE)      # 15 query imgs
_logits, _aloss, _mets = _model(_s_imgs, _s_lbl, _q_imgs)
print(f"Logits shape : {_logits.shape}")              # (15, 5)
print(f"Aux loss     : {_aloss.item():.4f}")
print(f"Density      : {_mets['graph_density']:.3f}")
n_param = sum(p.numel() for p in _model.parameters())
print(f"Total params : {n_param:,}")

## Section 12: Training Loop with Checkpointing

In [ ]:
# ── Section 12: Training Loop with Checkpointing ─────────────────────────────
import time, json
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm

# ── Hyperparameters (edit via config dict for ablations) ─────────────────────
CFG = {
    "n_way"          : 5,
    "k_shot"         : 1,
    "n_query"        : 15,
    "embed_dim"      : 128,
    "n_gnn_layers"   : 3,
    "rank"           : 16,
    "sparsity_mode"  : "l1",    # 'l1' | 'topk' | 'laplacian' | 'none'
    "lambda_sparse"  : 0.01,
    "lambda_edge"    : 0.5,
    "topk_k"         : 5,
    "lr"             : 5e-4,
    "weight_decay"   : 1e-5,
    "n_episodes_train": 300,     # episodes per epoch
    "n_epochs"       : 100,
    "t_warm"         : 5,        # warm-up epochs with static k-NN
    "edge_loss_ramp" : 20,       # epochs over which lambda_edge ramps to final
    "grad_clip"      : 1.0,
    "eval_every"     : 5,        # validate every N epochs
    "n_eval_episodes": 200,
    "seed"           : 42,
}


def build_model(cfg: dict) -> DEKAEModel:
    return DEKAEModel(
        embed_dim    = cfg["embed_dim"],
        n_gnn_layers = cfg["n_gnn_layers"],
        rank         = cfg["rank"],
        sparsity_mode= cfg["sparsity_mode"],
        lambda_sparse= cfg["lambda_sparse"],
        lambda_edge  = cfg["lambda_edge"],
        topk_k       = cfg["topk_k"],
        n_way        = cfg["n_way"],
    ).to(DEVICE)


def run_episode(model: DEKAEModel, episode_fn, n_way, k_shot, n_query,
                lambda_edge_scale: float = 1.0):
    """
    Runs one episode through the model and returns
    (loss, accuracy, metrics_dict).
    """
    s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
    s_imgs = s_imgs.to(DEVICE)
    s_lbl  = s_lbl.to(DEVICE)
    q_imgs = q_imgs.to(DEVICE)
    q_lbl  = q_lbl.to(DEVICE)

    logits, aux_loss, metrics = model(s_imgs, s_lbl, q_imgs, lambda_edge_scale)

    cls_loss = F.cross_entropy(logits, q_lbl)
    total    = cls_loss + aux_loss

    preds   = logits.argmax(dim=1)
    acc     = (preds == q_lbl).float().mean().item()
    return total, acc, metrics


def train(cfg: dict, episode_fn_train, episode_fn_val,
          run_name: str = "dekae", log_wandb: bool = False):
    """
    Full episodic training loop.

    episode_fn_train / episode_fn_val : callables (n_way, k_shot, n_query)
        returning (support_imgs, support_labels, query_imgs, query_labels)
    """
    set_seed(cfg["seed"])
    model     = build_model(cfg)
    optimizer = Adam(model.parameters(), lr=cfg["lr"],
                     weight_decay=cfg["weight_decay"])
    scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

    best_val_acc = 0.0
    history      = []

    for epoch in range(1, cfg["n_epochs"] + 1):
        t_start = time.time()

        # ── Warm-up: disable dynamic topology for first T_warm epochs ────────
        model.set_use_dynamic(epoch > cfg["t_warm"])

        # ── Curriculum lambda_edge scaling ────────────────────────────────────
        ramp_epochs      = cfg["edge_loss_ramp"]
        lambda_edge_scale = min(1.0, (epoch - cfg["t_warm"]) / max(ramp_epochs, 1))
        lambda_edge_scale = max(0.0, lambda_edge_scale)

        # ── Training episodes ─────────────────────────────────────────────────
        model.train()
        train_losses, train_accs = [], []
        grad_norms               = []

        pbar = tqdm(range(cfg["n_episodes_train"]),
                    desc=f"Ep {epoch}/{cfg['n_epochs']}", leave=False)
        for _ in pbar:
            optimizer.zero_grad()
            loss, acc, mets = run_episode(
                model, episode_fn_train,
                cfg["n_way"], cfg["k_shot"], cfg["n_query"],
                lambda_edge_scale
            )
            loss.backward()

            # Gradient clipping (Risk 7 mitigation)
            gnorm = nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
            grad_norms.append(gnorm)
            optimizer.step()
            train_losses.append(loss.item())
            train_accs.append(acc)
            pbar.set_postfix(loss=f"{loss.item():.3f}", acc=f"{acc:.3f}",
                             density=f"{mets['graph_density']:.2f}")

        scheduler.step()

        # ── Validation ────────────────────────────────────────────────────────
        val_acc_mean = 0.0
        if epoch % cfg["eval_every"] == 0:
            model.eval()
            val_accs = []
            with torch.no_grad():
                for _ in range(cfg["n_eval_episodes"]):
                    _, acc, _ = run_episode(
                        model, episode_fn_val,
                        cfg["n_way"], cfg["k_shot"], cfg["n_query"])
                    val_accs.append(acc)
            val_acc_mean = float(np.mean(val_accs))
            val_ci       = 1.96 * float(np.std(val_accs)) / (len(val_accs) ** 0.5)

            # ── Checkpoint best model → Drive ─────────────────────────────────
            if val_acc_mean > best_val_acc:
                best_val_acc = val_acc_mean
                ckpt_path    = CKPT_DIR / f"{run_name}_best.pth"
                torch.save({
                    "epoch"   : epoch,
                    "state"   : model.state_dict(),
                    "val_acc" : best_val_acc,
                    "cfg"     : cfg,
                }, str(ckpt_path))
                print(f"  ✓ New best val acc = {best_val_acc:.4f}  (ckpt saved)")

        epoch_time = time.time() - t_start
        log = {
            "epoch"           : epoch,
            "train_loss"      : float(np.mean(train_losses)),
            "train_acc"       : float(np.mean(train_accs)),
            "val_acc"         : val_acc_mean,
            "grad_norm_mean"  : float(np.mean(grad_norms)),
            "lambda_edge_scale": lambda_edge_scale,
            "use_dynamic"     : model.use_dynamic,
            "graph_density"   : mets.get("graph_density", 0),
            "avg_degree"      : mets.get("avg_degree", 0),
            "edge_entropy"    : mets.get("edge_entropy", 0),
            "layer_stability" : mets.get("layer_stability", []),
            "epoch_time_s"    : epoch_time,
        }
        history.append(log)

        # JSON backup to Drive every epoch
        with open(str(RESULTS_DIR / f"{run_name}_history.json"), "w") as f:
            json.dump(history, f)

        if log_wandb:
            import wandb
            wandb.log(log)

        if epoch % cfg["eval_every"] == 0:
            print(f"Epoch {epoch:4d} | train_acc={log['train_acc']:.4f} "
                  f"val_acc={val_acc_mean:.4f}±{val_ci:.4f} "
                  f"density={log['graph_density']:.3f} "
                  f"t={epoch_time:.1f}s")

    print(f"\nTraining complete. Best val acc: {best_val_acc:.4f}")
    return model, history


print("Training loop defined. Call train(CFG, ...) to start.")
print("Model parameter count:", sum(p.numel() for p in build_model(CFG).parameters()))

## Section 13: Evaluation & Statistical Testing

In [ ]:
# ── Section 13: Evaluation & Statistical Testing ─────────────────────────────
from scipy import stats as scipy_stats

def evaluate(model: DEKAEModel, episode_fn, n_way: int, k_shot: int,
             n_query: int, n_episodes: int = 1000) -> dict:
    """
    Evaluate over n_episodes test episodes.
    Returns accuracy mean, 95% CI, and full per-episode result dict.
    """
    model.eval()
    accs, densities, avg_degrees = [], [], []
    intra_ratios, inter_ratios   = [], []

    with torch.no_grad():
        for _ in tqdm(range(n_episodes), desc="Evaluating"):
            s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
            s_imgs = s_imgs.to(DEVICE)
            s_lbl  = s_lbl.to(DEVICE)
            q_imgs = q_imgs.to(DEVICE)
            q_lbl  = q_lbl.to(DEVICE)

            logits, _, mets = model(s_imgs, s_lbl, q_imgs)
            preds = logits.argmax(dim=1)
            acc   = (preds == q_lbl).float().mean().item()
            accs.append(acc)
            densities.append(mets["graph_density"])
            avg_degrees.append(mets["avg_degree"])

    accs_np  = np.array(accs)
    mean_acc = accs_np.mean()
    ci_95    = 1.96 * accs_np.std() / (len(accs_np) ** 0.5)

    return {
        "mean_acc"    : mean_acc,
        "ci_95"       : ci_95,
        "per_episode" : accs,
        "avg_density" : np.mean(densities),
        "avg_degree"  : np.mean(avg_degrees),
    }


def paired_ttest(accs_a: list, accs_b: list) -> dict:
    """
    Paired t-test between two models across the same episodes.
    Returns t-statistic and p-value. p < 0.05 → significant improvement.
    """
    t_stat, p_val = scipy_stats.ttest_rel(accs_b, accs_a)
    return {"t_statistic": t_stat, "p_value": p_val,
            "significant_at_0.05": p_val < 0.05}


def compute_intra_inter_edge_ratio(A: torch.Tensor, labels: torch.Tensor):
    """
    Given an adjacency matrix and node labels, compute the ratio of
    intra-class and inter-class edges (for graph quality reporting).
    """
    N = A.size(0)
    intra = inter = 0
    for i in range(N):
        for j in range(N):
            if i != j and A[i, j] > 0:
                if labels[i] == labels[j]:
                    intra += 1
                else:
                    inter += 1
    total = intra + inter
    return intra / max(total, 1), inter / max(total, 1)


# ── Example usage (illustrative — replace dummy episode_fn with real loader) ──
def _dummy_episode_fn(n_way, k_shot, n_query):
    """Placeholder using synthetic data — replace with real dataset loader."""
    s_imgs  = torch.randn(n_way * k_shot,  3, 84, 84)
    s_lbl   = torch.repeat_interleave(torch.arange(n_way), k_shot)
    q_imgs  = torch.randn(n_way * n_query, 3, 84, 84)
    q_lbl   = torch.repeat_interleave(torch.arange(n_way), n_query)
    return s_imgs, s_lbl, q_imgs, q_lbl

_eval_result = evaluate(_model, _dummy_episode_fn, n_way=5, k_shot=1,
                        n_query=15, n_episodes=50)
print(f"Accuracy: {_eval_result['mean_acc']*100:.2f} ± "
      f"{_eval_result['ci_95']*100:.2f}% (95% CI)")
print(f"Avg density: {_eval_result['avg_density']:.3f}")

## Section 14: Synthetic Graph Recovery Experiment (Section 2.3)

Planted-partition protocol: noise level $\sigma$ calibrated so k-NN achieves ~65% Graph F1, leaving room for DEKAE to demonstrate genuine topology recovery.

In [ ]:
# ── Section 14: Synthetic Graph Recovery Experiment ──────────────────────────
from sklearn.metrics import precision_score, recall_score, f1_score

def topology_recovery_metrics(A_pred: torch.Tensor,
                               A_star: torch.Tensor) -> dict:
    """
    Compare predicted adjacency against ground-truth planted adjacency.
    Binarises A_pred at 0.5 threshold.
    """
    A_pred_bin = (A_pred > 0.5).float()
    # Remove diagonal
    N = A_pred.size(0)
    mask = 1 - torch.eye(N, device=A_pred.device)
    p = A_pred_bin[mask.bool()].cpu().numpy()
    g = A_star[mask.bool()].cpu().numpy()

    precision = precision_score(g, p, zero_division=0)
    recall    = recall_score(g, p, zero_division=0)
    f1        = f1_score(g, p, zero_division=0)
    return {"edge_precision": precision, "edge_recall": recall, "graph_f1": f1}


def knn_recovery_baseline(X: torch.Tensor, A_star: torch.Tensor, k: int):
    """Run static k-NN and measure topology recovery."""
    A_knn = build_knn_adjacency(X, k=k)
    return topology_recovery_metrics(A_knn, A_star)


def synthetic_recovery_experiment(n_way=5, n_per_class=5, feat_dim=64,
                                   sigma_values=(0.3, 0.6, 0.8, 1.0),
                                   n_trials=50, knn_k=5):
    """
    Run the topology-recovery experiment across multiple noise levels.
    Returns a pandas DataFrame for easy tabulating.
    """
    try:
        import pandas as pd
        results = []
        for sigma in sigma_values:
            knn_f1s, ours_f1s = [], []
            for _ in range(n_trials):
                X, y, A_star = synthetic_episode(
                    n_way=n_way, n_nodes_per_class=n_per_class,
                    feat_dim=feat_dim, sigma=sigma)

                # k-NN baseline
                knn_met   = knn_recovery_baseline(X, A_star, k=knn_k)
                knn_f1s.append(knn_met["graph_f1"])

                # DEKAE topology recovery (single forward pass, no labels – unsupervised topology signal)
                A_init  = build_knn_adjacency(X, k=knn_k)
                eim     = EdgeIncidenceModule(feat_dim, feat_dim).to(X.device)
                dtm     = DynamicTopologyModule(feat_dim, rank=8, edge_dim=feat_dim).to(X.device)
                E_, src_, dst_ = eim(X, A_init)
                A_dek, _ = dtm(X, E_, src_, dst_, sparsity_reg=False)
                ours_met  = topology_recovery_metrics(A_dek.detach(), A_star)
                ours_f1s.append(ours_met["graph_f1"])

            results.append({
                "sigma"      : sigma,
                "kNN_F1_mean": round(np.mean(knn_f1s), 4),
                "kNN_F1_std" : round(np.std(knn_f1s), 4),
                "DEKAE_F1_mean": round(np.mean(ours_f1s), 4),
                "DEKAE_F1_std" : round(np.std(ours_f1s), 4),
            })

        df = pd.DataFrame(results)
        print(df.to_string(index=False))
        return df
    except ImportError:
        print("pandas not available — run: !pip install pandas")
        return None


# ── Run synthetic experiment ──────────────────────────────────────────────────
print("Running synthetic topology-recovery experiment …")
df_recovery = synthetic_recovery_experiment(
    n_way=5, n_per_class=5, feat_dim=64,
    sigma_values=[0.3, 0.6, 0.8, 1.0],
    n_trials=30, knn_k=5
)

## Section 15: Ablation Study Runner

Each group (A–G from the plan) can be run by calling `run_ablation(cfg_dict)`. Configs are defined per variant and results collected into a comparison table.

In [ ]:
# ── Section 15: Ablation Study Runner ────────────────────────────────────────
import copy

# ── Base config (derived from CFG) ───────────────────────────────────────────
BASE = copy.deepcopy(CFG)
BASE["n_epochs"]          = 30    # shorter for ablation sweeps
BASE["n_episodes_train"]  = 100
BASE["n_eval_episodes"]   = 100

# ──────────────────────────────────────────────────────────────────────────────
# Group A: Topology Dynamics
# ──────────────────────────────────────────────────────────────────────────────
ABLATION_CONFIGS = {

    # A1 — FSAKE baseline (static k-NN, no edge loss)
    "A1_FSAKE": {**BASE,
        "use_dynamic": False, "lambda_edge": 0.0, "sparsity_mode": "none"},

    # A2 — Dynamic rewiring only (no edge-to-node loss)
    "A2_dyn_only": {**BASE,
        "use_dynamic": True, "lambda_edge": 0.0},

    # A3 — Dynamic + edge-to-node projection (no edge loss)
    "A3_dyn_proj": {**BASE,
        "use_dynamic": True, "lambda_edge": 0.0},

    # A4 — Full DEKAE model
    "A4_DEKAE_full": {**BASE,
        "use_dynamic": True, "lambda_edge": 0.5},

    # ── Group B: Variable Degree ──────────────────────────────────────────────
    "B1_fixed_k5":  {**BASE, "use_dynamic": False, "topk_k": 5,  "lambda_edge": 0.0},
    "B2_fixed_k10": {**BASE, "use_dynamic": False, "topk_k": 10, "lambda_edge": 0.0},
    "B3_dynamic":   {**BASE, "use_dynamic": True,  "lambda_edge": 0.5},

    # ── Group E: Sparsity ─────────────────────────────────────────────────────
    "E1_no_sparse":   {**BASE, "sparsity_mode": "none"},
    "E2_l1":          {**BASE, "sparsity_mode": "l1"},
    "E3_topk":        {**BASE, "sparsity_mode": "topk"},
    "E4_laplacian":   {**BASE, "sparsity_mode": "laplacian"},

    # ── Group F: Edge MLP Capacity ────────────────────────────────────────────
    # (rank controls capacity; full MLP via larger hidden handled in model)
    "F1_full_rank64": {**BASE, "rank": 64},
    "F2_lowrank16":   {**BASE, "rank": 16},
    "F3_dotprod1":    {**BASE, "rank": 1},

    # ── Group G4: Lambda_edge Sensitivity ────────────────────────────────────
    "G4_lam0":   {**BASE, "lambda_edge": 0.0},
    "G4_lam01":  {**BASE, "lambda_edge": 0.1},
    "G4_lam05":  {**BASE, "lambda_edge": 0.5},
    "G4_lam1":   {**BASE, "lambda_edge": 1.0},
    "G4_lam2":   {**BASE, "lambda_edge": 2.0},
}


def run_ablation(name: str, cfg: dict, episode_fn_train, episode_fn_val,
                 log_wandb: bool = False):
    """
    Train and evaluate a single ablation variant.
    Returns a summary dict with name, mean_acc, ci_95.
    """
    print(f"\n{'='*60}")
    print(f"Running ablation: {name}")
    print(f"{'='*60}")
    set_seed(cfg.get("seed", 42))

    model  = build_model(cfg)
    model.set_use_dynamic(cfg.get("use_dynamic", True))

    # quick train
    trained_model, history = train(
        cfg, episode_fn_train, episode_fn_val,
        run_name=name, log_wandb=log_wandb
    )

    # evaluate
    res = evaluate(trained_model, episode_fn_val,
                   cfg["n_way"], cfg["k_shot"], cfg["n_query"],
                   n_episodes=cfg.get("n_eval_episodes", 200))
    res["name"] = name
    print(f"→ {name}: {res['mean_acc']*100:.2f} ± {res['ci_95']*100:.2f}%")
    return res


def run_all_ablations(groups: list, episode_fn_train, episode_fn_val):
    """Run a subset of ablation groups and collect results."""
    results = []
    for name in groups:
        if name not in ABLATION_CONFIGS:
            print(f"⚠ Unknown ablation key: {name}")
            continue
        res = run_ablation(name, ABLATION_CONFIGS[name],
                           episode_fn_train, episode_fn_val)
        results.append(res)
    return results


print("Ablation configs defined.")
print("Group A variants :", [k for k in ABLATION_CONFIGS if k.startswith("A")])
print("Group E variants :", [k for k in ABLATION_CONFIGS if k.startswith("E")])
print("Group G4 variants:", [k for k in ABLATION_CONFIGS if k.startswith("G4")])

## Section 16: Metrics Logging (Weights & Biases)

W&B is the recommended logger to survive Colab session disconnects. A JSON backup is saved to Drive at the end of every epoch as a fallback.

In [ ]:
# ── Section 16: Metrics Logging (W&B) ────────────────────────────────────────
# Run this cell to initialise W&B before calling train().
# Set LOG_WANDB=True in train() call to enable cloud sync.

try:
    import wandb

    WANDB_PROJECT = "DEKAE-FSL"
    WANDB_ENTITY  = None          # ← set to your W&B username/team if needed

    def init_wandb(run_name: str = "dekae_run", cfg: dict = None):
        wandb.init(
            project = WANDB_PROJECT,
            entity  = WANDB_ENTITY,
            name    = run_name,
            config  = cfg or CFG,
            resume  = "allow",    # resume if run_id matches previous session
        )
        print(f"W&B run initialised: {wandb.run.url}")

    # ── Full metrics schema (used by training loop) ───────────────────────────
    METRIC_SCHEMA = {
        "train_loss"         : None,   "train_acc"          : None,
        "val_acc"            : None,   "test_acc"           : None,
        "avg_degree"         : None,   "degree_std"         : None,
        "edge_entropy"       : None,   "graph_density"      : None,
        "topology_stability" : None,   "intra_edge_ratio"   : None,
        "inter_edge_ratio"   : None,   "layer_repr_stability": None,
        "grad_norm_mean"     : None,   "num_parameters"     : None,
        "epoch_time_s"       : None,   "lambda_edge_scale"  : None,
    }

    print("W&B available. Call init_wandb('run_name') before train().")
    print("Metric keys:", list(METRIC_SCHEMA.keys()))

except ImportError:
    print("W&B not installed. Fallback: JSON logs saved to:", RESULTS_DIR)

# ── Session-disconnect guard: flush & save ────────────────────────────────────
def safe_flush(history: list, run_name: str):
    """Save history to JSON and flush W&B (if active). Call from training loop."""
    path = RESULTS_DIR / f"{run_name}_history.json"
    with open(str(path), "w") as f:
        json.dump(history, f, default=str)
    try:
        import wandb
        if wandb.run is not None:
            wandb.save(str(path))
    except Exception:
        pass


print("safe_flush() registered.")

## Section 17: Visualization Suite

Five required plots: (1) topology per GNN layer, (2) degree distribution, (3) graph density curve, (4) t-SNE before/after, (5) degree vs. sample difficulty using margin difficulty $d_i = s_1(i) - s_2(i)$.

In [ ]:
# ── Section 17: Visualization Suite ──────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import networkx as nx
from sklearn.manifold import TSNE

sns.set_theme(style="whitegrid", font_scale=1.1)
SAVE_DIR = RESULTS_DIR / "figures"
SAVE_DIR.mkdir(exist_ok=True)


# ── Plot 1: Graph Topology per GNN Layer ─────────────────────────────────────

def plot_topology(A: torch.Tensor, labels: torch.Tensor, title: str = "",
                  save_path=None):
    """Visualise an episode graph coloured by class label."""
    N = A.size(0)
    A_np = (A > 0.01).float().cpu().numpy()
    G    = nx.from_numpy_array(A_np)

    label_list = labels.cpu().tolist()
    cmap = plt.cm.get_cmap("tab10", max(label_list) + 1)
    colors = [cmap(l) for l in label_list]

    fig, ax = plt.subplots(figsize=(6, 5))
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx(G, pos, node_color=colors,
                     with_labels=False, node_size=120,
                     edge_color="#aaa", width=0.6, ax=ax)
    ax.set_title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 2: Degree Distribution Histogram ─────────────────────────────────────

def plot_degree_distribution(A: torch.Tensor, title: str = "", save_path=None):
    A_no_diag = A.clone(); A_no_diag.fill_diagonal_(0)
    degrees = (A_no_diag > 0).float().sum(dim=1).cpu().numpy()
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.hist(degrees, bins=range(int(degrees.max()) + 2),
            color="steelblue", edgecolor="white")
    ax.set_xlabel("Node Degree")
    ax.set_ylabel("Count")
    ax.set_title(title or "Degree Distribution")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 3: Graph Density Curve over Training ─────────────────────────────────

def plot_density_curve(history: list, save_path=None):
    epochs    = [h["epoch"] for h in history]
    densities = [h.get("graph_density", 0) for h in history]
    fig, ax   = plt.subplots(figsize=(6, 3))
    ax.plot(epochs, densities, color="coral", linewidth=2)
    ax.axhline(0.5, ls="--", color="red", alpha=0.6, label="Collapse threshold (0.5)")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Graph Density")
    ax.set_title("Graph Density over Training (monitor for collapse)")
    ax.legend()
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 4: t-SNE Before / After Dynamic Topology ────────────────────────────

def plot_tsne(H_before: torch.Tensor, H_after: torch.Tensor,
              labels: torch.Tensor, save_path=None):
    H_all  = torch.cat([H_before, H_after], dim=0).cpu().detach().numpy()
    labels_np = labels.cpu().numpy()
    tsne   = TSNE(n_components=2, perplexity=15, random_state=42)
    emb    = tsne.fit_transform(H_all)
    N      = H_before.size(0)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for ax, title, start, end in zip(
            axes,
            ["Before (H⁰)", "After Dynamic Topology (Hᴸ)"],
            [0, N], [N, 2 * N]):
        sc = ax.scatter(emb[start:end, 0], emb[start:end, 1],
                        c=labels_np, cmap="tab10", s=40, alpha=0.8)
        ax.set_title(title); ax.axis("off")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Plot 5: Node Degree vs. Sample Difficulty ─────────────────────────────────

def plot_degree_vs_difficulty(model: DEKAEModel, episode_fn,
                               n_way: int, k_shot: int, n_query: int,
                               n_episodes: int = 600, save_path=None):
    """
    Bin query nodes into easy / medium / hard tertiles by margin difficulty
    d_i = s1(i) - s2(i)  (smaller margin = harder sample).
    Show median node degree per bin with 95% CI error bars.
    """
    model.eval()
    margin_diffs, degrees = [], []

    with torch.no_grad():
        for _ in tqdm(range(n_episodes), desc="Difficulty analysis"):
            s_imgs, s_lbl, q_imgs, q_lbl = episode_fn(n_way, k_shot, n_query)
            s_imgs = s_imgs.to(DEVICE); s_lbl = s_lbl.to(DEVICE)
            q_imgs = q_imgs.to(DEVICE)

            logits, _, mets = model(s_imgs, s_lbl, q_imgs)

            # Margin difficulty: difference between top-2 cosine similarities
            sorted_logits, _ = logits.sort(dim=1, descending=True)
            margin = (sorted_logits[:, 0] - sorted_logits[:, 1]).cpu().numpy()
            margin_diffs.extend(margin.tolist())

            # Per-query node degree from last A' (approximate via avg_degree here)
            # For a precise version, expose per-node degree from forward pass
            avg_d = mets.get("avg_degree", 0)
            degrees.extend([avg_d] * logits.size(0))

    margin_arr  = np.array(margin_diffs)
    degree_arr  = np.array(degrees)

    tertile1, tertile2 = np.percentile(margin_arr, [33, 66])
    bins = {"Hard\n(low margin)"  : degree_arr[margin_arr <= tertile1],
            "Medium"              : degree_arr[(margin_arr > tertile1) & (margin_arr <= tertile2)],
            "Easy\n(high margin)" : degree_arr[margin_arr > tertile2]}

    labels_plot  = list(bins.keys())
    means  = [np.median(v) for v in bins.values()]
    cis    = [1.96 * np.std(v) / max(np.sqrt(len(v)), 1) for v in bins.values()]

    fig, ax = plt.subplots(figsize=(5, 4))
    ax.bar(labels_plot, means, yerr=cis, color=["#e74c3c", "#f39c12", "#2ecc71"],
           capsize=6, edgecolor="white")
    ax.set_ylabel("Average Node Degree")
    ax.set_title("Node Degree vs. Sample Difficulty\n(Hard samples → more neighbours)")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=120)
    plt.show()


# ── Demo: run all plots on the toy model ─────────────────────────────────────

# Plot 1 — topology
_labels_demo = torch.cat([torch.full((4,), c) for c in range(5)]).to(DEVICE)
plot_topology(_A_dyn, _labels_demo, title="Episode Topology (Layer 3)",
              save_path=str(SAVE_DIR / "topology_layer3.png"))

# Plot 2 — degree distribution
plot_degree_distribution(_A_dyn, "Degree Distribution (DEKAE)",
                         save_path=str(SAVE_DIR / "degree_dist.png"))

# Plot 3 — density curve (requires history from train())
# Uncomment after running train():
# plot_density_curve(history, save_path=str(SAVE_DIR / "density_curve.png"))

# Plot 4 — t-SNE (requires H_before, H_after, labels from a real forward pass)
print("Plots 3/4/5 require training history & real data — run after train().")